#  Seleção de Atributos com ChiSqSelector (qui-quadrado)
### Inicializando o PySpark

In [14]:
import findspark
import pyspark
from pyspark.sql import SparkSession

# Faz a Interafce entre o Spark e o Jupyter Notebook
findspark.init()

# Inicializando uma Sessão no Spark
spark = SparkSession.builder.appName("ChiSqSelector").getOrCreate()

### Carregando a Base de Dados usado nesta Aula

In [15]:
carros = spark.read.load("../Material_do_Curso/Carros.csv", format="csv",
                        sep=";", inferSchema=True, header=True)
carros.show(5)

+-------+---------+-----------+---------------+----+-----+---------+-----------+-------+-----------+---+
|Consumo|Cilindros|Cilindradas|RelEixoTraseiro|Peso|Tempo|TipoMotor|Transmissao|Marchas|Carburadors| HP|
+-------+---------+-----------+---------------+----+-----+---------+-----------+-------+-----------+---+
|     21|        6|        160|             39| 262| 1646|        0|          1|      4|          4|110|
|     21|        6|        160|             39|2875| 1702|        0|          1|      4|          4|110|
|    228|        4|        108|            385| 232| 1861|        1|          1|      4|          1| 93|
|    214|        6|        258|            308|3215| 1944|        1|          0|      3|          1|110|
|    187|        8|        360|            315| 344| 1702|        0|          0|      3|          2|175|
+-------+---------+-----------+---------------+----+-----+---------+-----------+-------+-----------+---+
only showing top 5 rows



---

### Aplicando o RFurmula ao Conjunto de Dados Carros 

In [16]:
from pyspark.ml.feature import RFormula

In [17]:
# Instanciando um Objeto RFormula e aplicando a transformação dos dados
rformula = RFormula(formula="HP ~ .",
                    featuresCol="independente", labelCol="dependente")
carros_rf = rformula.fit(carros).transform(carros)
carros_rf.select("independente", "dependente").show(10, truncate=False)

+-----------------------------------------------------+----------+
|independente                                         |dependente|
+-----------------------------------------------------+----------+
|[21.0,6.0,160.0,39.0,262.0,1646.0,0.0,1.0,4.0,4.0]   |110.0     |
|[21.0,6.0,160.0,39.0,2875.0,1702.0,0.0,1.0,4.0,4.0]  |110.0     |
|[228.0,4.0,108.0,385.0,232.0,1861.0,1.0,1.0,4.0,1.0] |93.0      |
|[214.0,6.0,258.0,308.0,3215.0,1944.0,1.0,0.0,3.0,1.0]|110.0     |
|[187.0,8.0,360.0,315.0,344.0,1702.0,0.0,0.0,3.0,2.0] |175.0     |
|[181.0,6.0,225.0,276.0,346.0,2022.0,1.0,0.0,3.0,1.0] |105.0     |
|[143.0,8.0,360.0,321.0,357.0,1584.0,0.0,0.0,3.0,4.0] |245.0     |
|[244.0,4.0,1467.0,369.0,319.0,20.0,1.0,0.0,4.0,2.0]  |62.0      |
|[228.0,4.0,1408.0,392.0,315.0,229.0,1.0,0.0,4.0,2.0] |95.0      |
|[192.0,6.0,1676.0,392.0,344.0,183.0,1.0,0.0,4.0,4.0] |123.0     |
+-----------------------------------------------------+----------+
only showing top 10 rows



---

# Realizando a Seleção de Atributos com ChiSqSelector

Seleção de atributos: técnica de engenharia de atributos que busca melhorar a performance do modelo através da
seleção de um subconjunto de atributos mais relevantes para prever a classe.

Usa o teste de independência do qui-quadrado para selecionar os atributos.

Recebe uma coluna com um vetor de atributos e produz uma coluna com um vetor de atributos mais relevantes

Parâmetros:

> + **percentile**: percentual de atributos que devem ser selecionados. Padrão 0.1
> + **fpr e fwe**: seleciona atributos que os valores- p estejam abaixo de um parâmetro. Padrão 0.05
> + **fdr**: aplica o critério de FDR (False Discovery Rate) . Padrão 0.05




In [18]:
from pyspark.ml.feature import ChiSqSelector

#### Usando o Parâmetro  ***fdr = 0.01***

In [24]:
selector = ChiSqSelector(selectorType="fdr", fdr=0.01, featuresCol="independente",
                        outputCol="selecionados", labelCol="dependente")
carros_cs = selector.fit(carros_rf).transform(carros_rf)
carros_cs.select("selecionados").show(10, truncate=False)

+--------------+
|selecionados  |
+--------------+
|[160.0,39.0]  |
|[160.0,39.0]  |
|[108.0,385.0] |
|[258.0,308.0] |
|[360.0,315.0] |
|[225.0,276.0] |
|[360.0,321.0] |
|[1467.0,369.0]|
|[1408.0,392.0]|
|[1676.0,392.0]|
+--------------+
only showing top 10 rows



#### Usando o Parâmetro  ***numTopFeatures = 2***

In [23]:
selector = ChiSqSelector(selectorType="numTopFeatures", numTopFeatures=4, featuresCol="independente",
                        outputCol="selecionados", labelCol="dependente")
carros_cs = selector.fit(carros_rf).transform(carros_rf)
carros_cs.select("selecionados").show(10, truncate=False)

+----------------------+
|selecionados          |
+----------------------+
|[6.0,160.0,39.0,4.0]  |
|[6.0,160.0,39.0,4.0]  |
|[4.0,108.0,385.0,1.0] |
|[6.0,258.0,308.0,1.0] |
|[8.0,360.0,315.0,2.0] |
|[6.0,225.0,276.0,1.0] |
|[8.0,360.0,321.0,4.0] |
|[4.0,1467.0,369.0,2.0]|
|[4.0,1408.0,392.0,2.0]|
|[6.0,1676.0,392.0,4.0]|
+----------------------+
only showing top 10 rows



---

# Seleção de Atributos com UnivariateFeatureSelector

Seleção de atributos

Você pode definir os tipos de atributos das variáveis dependentes e independentes que quer
usar (categórico ou contínuo)

De acordo com a definição, o Spark usa uma técnica de seleção de atributos

**Parâmetros**

> + **featureType**: Tipo das características, pode ser categorical ou continuous
> + **labelType**: Tipo da classe. pode ser categorical ou continuous
> + **selectionThreshold**: Limite máximo de características que serão selecionadas.




In [25]:
from pyspark.ml.feature import UnivariateFeatureSelector

In [27]:
selector_uni_var = UnivariateFeatureSelector(featuresCol="independente",
                                             outputCol="selecionados_uni_var",
                                            labelCol="dependente")
selector_uni_var.setFeatureType("continuous").setLabelType("continuous").setSelectionThreshold(5)
carros_univar = selector_uni_var.fit(carros_rf).transform(carros_rf)
carros_univar.select("selecionados_uni_var").show(10, truncate=False)

+-----------------------+
|selecionados_uni_var   |
+-----------------------+
|[21.0,6.0,0.0,1.0,4.0] |
|[21.0,6.0,0.0,1.0,4.0] |
|[228.0,4.0,1.0,1.0,1.0]|
|[214.0,6.0,1.0,0.0,1.0]|
|[187.0,8.0,0.0,0.0,2.0]|
|[181.0,6.0,1.0,0.0,1.0]|
|[143.0,8.0,0.0,0.0,4.0]|
|[244.0,4.0,1.0,0.0,2.0]|
|[228.0,4.0,1.0,0.0,2.0]|
|[192.0,6.0,1.0,0.0,4.0]|
+-----------------------+
only showing top 10 rows

